In [121]:
from urllib.request import urlopen
# from selenium import webdriver as wd 
from bs4 import BeautifulSoup
from tqdm import tqdm

import itertools
import pandas as pd
import numpy as np
import requests
import time
import copy
import re

## 현재 존재하는 아이템 리스트 가져오기

In [122]:
def make_itemlist():
    url = 'https://lolchess.gg/items'
    hdr = {'Accept-Language' : 'Ko_KR', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    req = requests.get(url, headers = hdr)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')

    itemdict = {}
    for i in soup.find_all('div',{'class':'col-sm-6'}): 
        arr = []
        for index, item in enumerate(i.select('img')):
            if index == 2:
                itemdict[item.get('alt')] = arr[:]
            else:
                arr.append(item.get('alt'))
        arr.clear()
    return itemdict

In [123]:
make_itemlist()

{'죽음의 검': ['B.F. 대검', 'B.F. 대검'],
 '거인 학살자': ['곡궁', 'B.F. 대검'],
 '수호 천사': ['쇠사슬 조끼', 'B.F. 대검'],
 '피바라기': ['음전자 망토', 'B.F. 대검'],
 '마법공학 총검': ['쓸데없이 큰 지팡이', 'B.F. 대검'],
 '쇼진의 창': ['여신의 눈물', 'B.F. 대검'],
 '지크의 전령': ['거인의 허리띠', 'B.F. 대검'],
 '신성의 검': ['뒤집개', 'B.F. 대검'],
 '무한의 대검': ['연습용 장갑', 'B.F. 대검'],
 '고속 연사포': ['곡궁', '곡궁'],
 '거인의 결의': ['쇠사슬 조끼', '곡궁'],
 '루난의 허리케인': ['음전자 망토', '곡궁'],
 '구인수의 격노검': ['쓸데없이 큰 지팡이', '곡궁'],
 '스태틱의 단검': ['여신의 눈물', '곡궁'],
 '즈롯 차원문': ['거인의 허리띠', '곡궁'],
 '결투가의 열정': ['뒤집개', '곡궁'],
 '최후의 속삭임': ['연습용 장갑', '곡궁'],
 '덤불 조끼': ['쇠사슬 조끼', '쇠사슬 조끼'],
 '가고일 돌갑옷': ['음전자 망토', '쇠사슬 조끼'],
 '강철의 솔라리 펜던트': ['쓸데없이 큰 지팡이', '쇠사슬 조끼'],
 '얼어붙은 심장': ['여신의 눈물', '쇠사슬 조끼'],
 '태양불꽃 망토': ['거인의 허리띠', '쇠사슬 조끼'],
 '수호자의 흉갑': ['뒤집개', '쇠사슬 조끼'],
 '침묵의 장막': ['연습용 장갑', '쇠사슬 조끼'],
 '용의 발톱': ['음전자 망토', '음전자 망토'],
 '이온 충격기': ['쓸데없이 큰 지팡이', '음전자 망토'],
 '힘의 성배': ['여신의 눈물', '음전자 망토'],
 '서풍': ['거인의 허리띠', '음전자 망토'],
 '나무정령 새싹': ['뒤집개', '음전자 망토'],
 '수은': ['연습용 장갑', '음전자 망토'],
 '라바돈의 죽음모자': ['쓸데없이 큰 지팡이', '쓸

## 아이템 리스트 불러오기

In [124]:
item_list = make_itemlist()
a = set()
for key, value in item_list.items():
    for i in value:
        a.add(i)
a = list(a)
a.sort()
small_itemlist=a

In [125]:
small_itemlist

['B.F. 대검',
 '거인의 허리띠',
 '곡궁',
 '뒤집개',
 '쇠사슬 조끼',
 '쓸데없이 큰 지팡이',
 '여신의 눈물',
 '연습용 장갑',
 '음전자 망토']

## 합친 아이템을 분해해서 구성요소를 볼 수 있는 함수

In [126]:
def decomposition_item(item_list,arr):
    small_item = []
    for full in arr:
        if full in item_list:
            small_item.append(item_list[full][0])
            small_item.append(item_list[full][1])
        else:
            small_item.append(full)
        small_item.sort()

    return small_item

## Beautiful Soup을 이용하여서 상위 티어의 아이템/ 시너지/ 승률 가져오기

In [255]:
# url을 이용하여서 html을 받아오는 함수
def rcv_sp(hdr, url, check):
    req = requests.get(url, headers = hdr)
    sp = BeautifulSoup(req.text, 'html.parser')
    
    if check == 1:
        return sp
    else : 
        return sp.find('div',{'class' : 'profile__match-history-v2__filters'}).select('a')[1]['href']

#해당 게임의 아이템을 받아오는 함수     
def rcv_items(itemsp):
    itemlist = []
    for i in itemsp:
        tp = i.select('.item')
        if len(tp) == 0:
            continue
        for items in tp:
            itemlist.append(items['title'])
    return itemlist

#해당 게임의 조합을 받아오는 함수
def rcv_traits(traitsp):
    traitslist = []
    for i in traitsp:
        traitslist.append(i.find('img')['alt'])
        
    return traitslist
    
#해당 게임의 챔프들을 받아오는 함수
def rcv_units(unitsp):
    unitlist = []
    mainunitlist = {}
    
    for unit in unitsp:
        it = []
        #champ : 챔피언만을 갖옴
        champ = unit.select('.tft-champion')[0].find('img')['alt']
        unitlist.append(champ)
        # mainitem : 챔피언이 아이템을 가진 경우를 보여줌 
        mainitem = unit.select('.items')[0].find_all('img')
        if len(mainitem) > 0:
            for con in mainitem:
                it.append(con['title'])
            mainunitlist[champ] = it[:]
        it.clear()
    return unitlist, mainunitlist


def print_list(arr):
    for i in arr:
        print(i)
    
def main_func():
    url = 'https://lolchess.gg/leaderboards'
    hdr = {'Accept-Language' : 'Ko_KR', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    req = requests.get(url, headers = hdr)
    soup = (BeautifulSoup(req.text,'html.parser'))
    #데이터를 저장할 배열
    save_data = []
    for i in tqdm(soup.find('tbody').select('a')):
        #처음에 i 에서 유저의 정보를 받아서 이 유저의 랭크의 soup를 받아온다
        sp = rcv_sp(hdr,rcv_sp(hdr,i['href'],0),1)
        #하나씩 저장
        temp_data = []
        for j in sp.select('.profile__match-history-v2__item'):
            #rank :  순위
            rank = j.find('div',{'class':'placement'}).text.strip()[1]
            #traits : 조합
            traits = rcv_traits(j.select('.tft-hexagon-image'))
            #unit : 챔프
            units, champitems = rcv_units(j.select('.unit'))
            print(champitems)
            
            #items : 아이템
            items = rcv_items(j.select('.items'))
            temp_data.append(rank)
            temp_data.append(traits)
            temp_data.append(items)
            temp_data.append(units)
            save_data.append(temp_data[:])
            temp_data.clear()
            break
        break
    
#         print_list(save_data)
    return save_data    
    

In [256]:
data = main_func()






  0%|          | 0/100 [00:01<?, ?it/s]

{'이렐리아': ['죽음의 저항', '용의 정령'], '쉔': ['란두인의 성소', '태양불꽃 망토', '구원'], '리 신': ['존야의 역설'], '세트': ['나무정령 새싹', '거인의 결의', '마법공학 총검'], '요네': ['영원한 겨울', '용의 발톱', '푸른 파수꾼'], '질리언': ['수호 천사', '쇼진의 창', '쇼진의 창']}


In [33]:
def find_meta(item_dict):

#     driver = wd.Chrome(executable_path="/mnt/c/Users/shurona/Documents/chromedriver.exe")
    url = 'https://lolchess.gg/meta'
#     driver.get(url)
    hdr = {'Accept-Language' : 'Ko_KR'}
    req = requests.get(url, headers = hdr)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    tp = soup.find("div",{"class" : "container-full"})
    
    #최종 저장 리스트
    final_set = []
    
    #각각의 내용을 저장할 장소
    title = {}
    Configuration = []
    champ_list = []
    items_list = []
    main_johap = []
    small_title = []
    #페이지의 메타를 정리해놓은 표에서 시너지 별로 표를 정리한다. 
    for i,v in enumerate(tp.select('div[class=guide-meta__deck]')):
        #sub 리스트 
        sub = []
        
        #공백 지우기 
        del_blank = v.find('div',{"class" : "guide-meta__deck__column name mr-3"}).text.strip()
    
        #데이터를 저장할 리스트들
        johap_name=''
        
        #조합 모으기
        for find_togle in v.select(".tft-hexagon-image"):
            num = (find_togle.get('data-tooltip-url')[-1])
            word = (find_togle.find('img', alt = True)['alt'])
            Configuration.append(str(num) + " " + word)
        
        #챔프 모으기 
        for find_champ in v.select(".tft-champion"):
            name = find_champ.find('img', alt=True)['alt']
            champ_list.append(name)

        #아이템 모으기
        for find_item in v.select(".tft-items"):
            t = find_item.select('img')
            if len(t) == 0:
                continue
            for trash in t:
                trash['alt']
        
        count = 0
        for index,value in enumerate(del_blank):
        
            if ord('가') <= ord(value) <= ord('힣'):
                if count == 0:
                    start = index
                count=1
            else:
                if count == 1:
                    johap_name = johap_name+(''.join(del_blank[start:index]))
                    count = 0
        if count == 1:
#             small_title.append(del_blank[start:])
            johap_name = johap_name+(del_blank[start:])
            
        small_title.append(johap_name)
            
        for item in v.find_all('div',{'class':'tft-items'}):
            items = item.find_all('img')
            if len(items) == 0:
                continue
            for tem in items:
                items_list.append(tem.get('alt'))
                
                
        sub.append(Configuration[:])
        sub.append(champ_list[:])
        sub.append(items_list[:])
        main_johap.append(sub[:])
#         title['구성'] = Configuration[:]
#         title['챔피언'] = champ_list[:]
#         title['아이템'] = items_list[:]

        final_set.append(title.copy())
        

        champ_list.clear()
        Configuration.clear()
        items_list.clear()
        title.clear()
        sub.clear()

    main_johap.append(small_title)
    return main_johap

In [34]:
final_set = find_meta(item_list)

In [35]:
for i in final_set[:-1]:
    #before : 아이템들의 리스트
    before = i.pop()
    #after : 아이템들 분해후 리스트
    after = decomposition_item(item_list,before)
    af = list(set(after))
    for index, value in enumerate(af):
        if value[0].isnumeric():
            break
        af_tp = af[index]
        af[index] = str(after.count(value))+af_tp
#     print(af)
    i.append(af)

In [36]:
final_set[-1]

['싸움꾼',
 '귀감',
 '요술사',
 '처형자신성',
 '처형자조율자신성',
 '광신도',
 '선봉신비우화',
 '영혼시비르',
 '선봉학살올라프',
 '나무정령',
 '결투가',
 '용의영혼',
 '흡수자선지자',
 '처형자나무정령',
 '대장군학살자',
 '학살자명사수',
 '흡수자신성']

## 아이템의 갯수를 비교해서 가장 근접한 조합을 넣는다. 

In [37]:
#final_set : 메인조합을 저장해 놓은 리스트
#temp : 메인조합을 찾고자하는 전적조합 [갯수+기본아이템]
def match_item(final_set, temp):
    tp = []
    max_score = 0
    now = -1
    for i, v in enumerate(final_set):
        score = 0
        total = 0
        for value in temp:
            #s : 해당 아이템의 갯수
            s = int(value[0])
            total += s
            item = value[1:]
            for it in v[2]:
#                 print(f'it : {it[1:]}, item : {item}')
            # v의 예시 ['3연습용 장갑', '3음전자 망토', '2곡궁', '2B.F. 대검', '2여신의 눈물', '4쓸데없이 큰 지팡이']]
                if it[1:] == item:
                    score += min(s,int(it[0]))
                    break
        if max_score < score:
            max_score = score
            now = i
    
    if now != -1:
        #final set의 마지막에는 조합들이 들어있다. 거기서 now를 찾아서 get 
        tp.append(final_set[-1][now])
        tp.append(round(max_score/total,2))
    else:
        tp.append(0)
        tp.append(0)
    
    return tp

In [38]:
index = 0
print(item_list.keys())
for i,v in item_list.items():
    index+=1
#     print(i)
# index
# item_list

dict_keys(['죽음의 검', '거인 학살자', '수호 천사', '피바라기', '마법공학 총검', '쇼진의 창', '지크의 전령', '신성의 검', '무한의 대검', '고속 연사포', '거인의 결의', '루난의 허리케인', '구인수의 격노검', '스태틱의 단검', '즈롯 차원문', '결투가의 열정', '최후의 속삭임', '덤불 조끼', '가고일 돌갑옷', '강철의 솔라리 펜던트', '얼어붙은 심장', '태양불꽃 망토', '수호자의 흉갑', '침묵의 장막', '용의 발톱', '이온 충격기', '힘의 성배', '서풍', '나무정령 새싹', '수은', '라바돈의 죽음모자', '루덴의 메아리', '모렐로노미콘', '용의 정령', '보석 건틀릿', '푸른 파수꾼', '구원', '요술사의 모자', '정의의 손길', '워모그의 갑옷', '대장군의 깃발', '덫 발톱', '대자연의 힘', '요우무의 유령검', '도적의 장갑'])


## 오른의 역설 제거

In [111]:
def remove_ornn(arr,itemlist):
    tp = itemlist.keys()
    arr_return = []
    for i in arr:
        if i in tp:
            arr_return.append(i)
    return arr_return  

## 근접한 조합을 찾기 위한 data가 도는 반복문

In [112]:
cp = copy.deepcopy(data)
# item_list : 아이템들을 저장해 놓은 dict 
# final_set : 메인조합을 담아놓은 dict 
for i in cp:
    #i[0] : rank // i[1] : 조합 // i[2] : 기존 아이템 // i[3] : 챔프 // i[4] 분해된 아이템
    full_item = i[2]
    
    #오른의 걸작 아이템 삭제
    full_item = remove_ornn(full_item,item_list)
    i[2] = full_item
    small_item = decomposition_item(item_list,full_item)
    traits = i[1]

    # sm : 아이템의 갯수를 확인하기 위해서 중복을 지운 기준 값을 정함
    sm = list(set(small_item))
    for index, value in enumerate(sm):
        #이미 분할 되었으면 break
        if value[0].isnumeric():
            break
        tp_1 = sm[index]
        sm[index]= (str(small_item.count(value)))+tp_1
    
    johab = match_item(final_set, sm)
    
    #정리된 값을 i에 추가
    i.append(sm)
    #닯은 정도를 넣어줌 
    i[2].append((int(johab[1]*100)))
    i.append(johab[0])
    i[1].sort(reverse=True)

In [113]:
# items : ['B.F. 대검', '거인의 허리띠', '곡궁', '뒤집개', '쇠사슬 조끼', '쓸데없이 큰 지팡이', '여신의 눈물', '연습용 장갑', '음전자 망토']
# arr 분해된 아이템
def make_itemlist(arr,items):
    item_ct = []
    for i in items:
        check = 0
        for j in arr:
#             print(j)
            ct = int(j[0])
            if i == j[1:]:
                item_ct.append(ct)
                check = 1
                break
        if check == 0:
            item_ct.append(0)
    return item_ct

## data를 csv로 만들기 위해 분류하는 반복문

In [114]:
# for j in cp[:5]:
#     for i in j:
#         print(i)
#     print()

In [115]:
# small_itemlist : ['B.F. 대검', '거인의 허리띠', '곡궁', '뒤집개', '쇠사슬 조끼', '쓸데없이 큰 지팡이', '여신의 눈물', '연습용 장갑', '음전자 망토']
#v[0] : 랭크 // v[1] : 세부조합 // v[2] : 아이템 v[2][-1] : 메인조합 유사도 // v[3] : 챔피온 // v[4] : 분해된 아이템 // v[5] : 조합 이름
#cp_basic 기존의 리스트
index = 0
cp_basic = []
for i,v in enumerate(cp):
    index += 1
    ar = make_itemlist(v[4],small_itemlist)
    #유사도를 옮기기 위해 pop
    usado = v[2][-1]
    # itemlist 배열에 rank 및 인덱스 추가 ==> index의 경우 외래 키로 사용 예정
    ar.insert(0,int(v[0]))
    ar.insert(0,index)
    ar.append(v[5])
    ar.append(usado)
#     print(ar)
    cp_basic.append(ar)
    v[0] = index
    v[2] = v[2][:-1]
#     break

In [116]:
for i in cp[0]:
    print(i)

1
['4 조율자', '3 나무정령', '2 싸움꾼', '2 신성', '2 신비술사', '1 황제', '1 추방자', '1 우두머리', '1 대장장이', '1 닌자']
['용의 정령', '태양불꽃 망토', '구원', '나무정령 새싹', '거인의 결의', '마법공학 총검', '용의 발톱', '푸른 파수꾼', '수호 천사', '쇼진의 창', '쇼진의 창']
['누누와 윌럼프', '이렐리아', '쉔', '리 신', '세트', '아지르', '오른', '요네', '질리언']
['2쓸데없이 큰 지팡이', '3음전자 망토', '2뒤집개', '5여신의 눈물', '1곡궁', '2거인의 허리띠', '4B.F. 대검', '3쇠사슬 조끼']
흡수자선지자


In [117]:
cp_basic[0]

[1, 2, 4, 2, 1, 2, 3, 2, 5, 0, 3, '흡수자선지자', 68]

In [120]:
ddddd = 0
for i in cp_basic:
    ddddd = max(ddddd,len(i[-2]))
    
print(ddddd)

8


## 1. itemlist배열 저장 과정

In [91]:
cp_basic = np.array(cp_basic)

In [88]:
columns = list(itertools.chain(['id','rank'],small_itemlist,['조합','유사도']))

In [89]:
df = pd.DataFrame(cp_basic,columns=columns)

In [90]:
df.head()

,id,rank,B.F. 대검,거인의 허리띠,곡궁,뒤집개,쇠사슬 조끼,쓸데없이 큰 지팡이,여신의 눈물,연습용 장갑,음전자 망토,조합,유사도
0,1,2,4,2,1,2,3,2,5,0,3,흡수자선지자,68
1,2,1,2,3,1,1,2,2,2,2,5,요술사,75
2,3,2,2,5,2,0,2,1,4,2,2,처형자나무정령,75
3,4,1,0,0,6,1,2,6,2,4,1,결투가,59
4,5,2,2,4,1,0,2,1,2,3,3,처형자나무정령,83


In [91]:
df.to_csv('itemlist_ver2.csv',index=False,encoding='utf-8-sig')

## 2. 조합 상세 저장 과정

In [92]:
cpp = np.array(cp)

In [93]:
cp_col = ["id","조합상세","사용된 아이템","사용된 챔프","분해된 아이템","조합이름"]

In [94]:
dfcp = pd.DataFrame(cpp,columns=cp_col)

In [95]:
# dfcp['사용된 아이템'].values

In [98]:
dfcp['조합상세'] = dfcp['조합상세'].str.join(',')
dfcp['사용된 아이템'] = dfcp['사용된 아이템'].str.join(',')
dfcp['사용된 챔프'] = dfcp['사용된 챔프'].str.join(',')
dfcp['분해된 아이템'] = dfcp['분해된 아이템'].str.join(',')

In [100]:
dfcp.head()

,id,조합상세,사용된 아이템,사용된 챔프,분해된 아이템,조합이름
0,1,"4 조율자,3 나무정령,2 싸움꾼,2 신성,2 신비술사,1 황제,1 추방자,1 우두...","용의 정령,태양불꽃 망토,구원,나무정령 새싹,거인의 결의,마법공학 총검,용의 발톱,...","누누와 윌럼프,이렐리아,쉔,리 신,세트,아지르,오른,요네,질리언","2쓸데없이 큰 지팡이,3음전자 망토,2뒤집개,5여신의 눈물,1곡궁,2거인의 허리띠,...",흡수자선지자
1,2,"7 요술사,3 용의 영혼,3 나무정령,2 흡수자,1 황제","루난의 허리케인,서풍,힘의 성배,정의의 손길,마법공학 총검,수호 천사,보석 건틀릿,...","브랜드,트위스티드 페이트,룰루,애니,베이가,모르가나,아우렐리온 솔,아지르,스웨인","2연습용 장갑,2쓸데없이 큰 지팡이,5음전자 망토,1뒤집개,2여신의 눈물,1곡궁,3...",요술사
2,3,"6 학살자,3 용의 영혼,2 선봉대,2 명사수,1 무모한 자,1 닌자","구원,힘의 성배,지크의 전령,침묵의 장막,워모그의 갑옷,모렐로노미콘,얼어붙은 심장,...","트리스타나,제드,브라움,파이크,다리우스,세주아니,올라프,사미라","2연습용 장갑,1쓸데없이 큰 지팡이,2음전자 망토,4여신의 눈물,2곡궁,5거인의 허...",처형자나무정령
3,4,"6 용의 영혼,4 선봉대,3 학살자,2 명사수,1 무모한 자,1 대장장이","도적의 장갑,스태틱의 단검,라바돈의 죽음모자,구인수의 격노검,구인수의 격노검,정의의...","트리스타나,브랜드,브라움,세주아니,아트록스,올라프,스웨인,오른,사미라","4연습용 장갑,6쓸데없이 큰 지팡이,1음전자 망토,1뒤집개,6곡궁,2여신의 눈물,2...",결투가
4,5,"7 대장군,2 암살자,2 싸움꾼,2 귀감,1 황제,1 추방자,1 우두머리","정의의 손길,서풍,태양불꽃 망토,덫 발톱,수은,마법공학 총검,거인의 결의,지크의 전...","니달리,파이크,바이,자르반 4세,카타리나,트린다미어,세트,아지르,아지르,요네","3연습용 장갑,1쓸데없이 큰 지팡이,3음전자 망토,2여신의 눈물,1곡궁,4거인의 허...",처형자나무정령


In [102]:
dfcp.to_csv('item_sebu_ver2.csv',index=False,encoding='utf-8-sig')

In [103]:
pd_tp = pd.read_csv('item_sebu_ver1.csv')

In [104]:
pd_tp.head()

,id,조합상세,사용된 아이템,사용된 챔프,분해된 아이템,조합이름
0,1,"4 조율자,3 나무정령,2 싸움꾼,2 신성,2 신비술사,1 황제,1 추방자,1 우두...","용의 정령,태양불꽃 망토,구원,나무정령 새싹,거인의 결의,마법공학 총검,용의 발톱,...","누누와 윌럼프,이렐리아,쉔,리 신,세트,아지르,오른,요네,질리언","4B.F. 대검,2뒤집개,2거인의 허리띠,2쓸데없이 큰 지팡이,3쇠사슬 조끼,1곡궁...",흡수자선지자
1,2,"7 요술사,3 용의 영혼,3 나무정령,2 흡수자,1 황제","루난의 허리케인,서풍,힘의 성배,정의의 손길,마법공학 총검,수호 천사,보석 건틀릿,...","브랜드,트위스티드 페이트,룰루,애니,베이가,모르가나,아우렐리온 솔,아지르,스웨인","2B.F. 대검,1뒤집개,3거인의 허리띠,2쓸데없이 큰 지팡이,2쇠사슬 조끼,1곡궁...",요술사
2,3,"6 학살자,3 용의 영혼,2 선봉대,2 명사수,1 무모한 자,1 닌자","구원,힘의 성배,지크의 전령,침묵의 장막,워모그의 갑옷,모렐로노미콘,얼어붙은 심장,...","트리스타나,제드,브라움,파이크,다리우스,세주아니,올라프,사미라","2B.F. 대검,5거인의 허리띠,1쓸데없이 큰 지팡이,2쇠사슬 조끼,2곡궁,2음전자...",처형자나무정령
3,4,"6 용의 영혼,4 선봉대,3 학살자,2 명사수,1 무모한 자,1 대장장이","도적의 장갑,스태틱의 단검,라바돈의 죽음모자,구인수의 격노검,구인수의 격노검,정의의...","트리스타나,브랜드,브라움,세주아니,아트록스,올라프,스웨인,오른,사미라","1뒤집개,6쓸데없이 큰 지팡이,2쇠사슬 조끼,6곡궁,1음전자 망토,2여신의 눈물,4...",결투가
4,5,"7 대장군,2 암살자,2 싸움꾼,2 귀감,1 황제,1 추방자,1 우두머리","정의의 손길,서풍,태양불꽃 망토,덫 발톱,수은,마법공학 총검,거인의 결의,지크의 전...","니달리,파이크,바이,자르반 4세,카타리나,트린다미어,세트,아지르,아지르,요네","2B.F. 대검,4거인의 허리띠,1쓸데없이 큰 지팡이,2쇠사슬 조끼,1곡궁,3음전자...",처형자나무정령


In [106]:
pd_numpy = pd_tp.to_numpy()

In [109]:
pd_list = pd_numpy.tolist()

In [110]:
pd_list[0]

[1,
 '4 조율자,3 나무정령,2 싸움꾼,2 신성,2 신비술사,1 황제,1 추방자,1 우두머리,1 대장장이,1 닌자',
 '용의 정령,태양불꽃 망토,구원,나무정령 새싹,거인의 결의,마법공학 총검,용의 발톱,푸른 파수꾼,수호 천사,쇼진의 창,쇼진의 창',
 '누누와 윌럼프,이렐리아,쉔,리 신,세트,아지르,오른,요네,질리언',
 '4B.F. 대검,2뒤집개,2거인의 허리띠,2쓸데없이 큰 지팡이,3쇠사슬 조끼,1곡궁,3음전자 망토,5여신의 눈물',
 '흡수자선지자']

## 셀레니움을 사용하여서 상위 티어의 전적 가져오기

In [8]:
#decks > div > section > div.col-12.col-xl-9

#soup를 받아오는 함수
def receive_soup(url, hdr):
    req = requests.get(url,headers = hdr)
    html = req.text
    return(BeautifulSoup(html, 'html.parser'))
    
#url을 받아오면 여기서 s
def get_info(url,hdr,driver):
    req = requests.get(url,headers = hdr)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    history = soup.find('div',{'class' : 'profile__match-history-v2__filters'}).select('a')[1]['href']
    url = history
    driver.get(url)
    for i in driver.find_elements_by_class_name('toggle-detail'):
        i.click()
        time.sleep(1)
    time.sleep(2)
    sp = BeautifulSoup(driver.page_source, 'html.parser')
    return sp
    
options = wd.ChromeOptions()
options.add_argument('--window-size=1920,1080')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = wd.Chrome(executable_path="/mnt/c/Users/shurona/Documents/chromedriver.exe", options=options)    
url = 'https://lolchess.gg/leaderboards'
hdr = {'Accept-Language' : 'Ko_KR', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
req = requests.get(url, headers = hdr)
html = req.text
soup = BeautifulSoup(html, 'html.parser')
for i in soup.find('tbody').select('a'):
    sp = get_info(i['href'],hdr,driver)
    break

In [12]:
import itertools
count  = 1
for i in sp.find_all('div',{'class' : 'profile__match-history-v2__item__detail'}):
    for score, value in enumerate(i.find('tbody').find_all('tr')):
        print(count)
        count += 1
        arr=[]
        need_item=[]

        traits = value.select_one('td[class=traits]')
        champions = value.select_one('td[class=champions]')  
        for kinds in traits.select('.tft-hexagon-image'):
            print(kinds.find('img')['alt'], end = ' ')
        print()
        for champ in champions.select('.champions__image'):
            print(champ.find('img')['alt'], end = ' ')
        print()
        for items in champions.select('.champions__items'):
            if items.find('img') != None:
                for i in items.select('img'):
                    arr.append(i['data-original-title'])
#         print(arr)
        for big in arr:
            if big in item_list.keys():
                for divide in item_list[big]:
                    need_item.append(divide)
        print(list(set(need_item)))
        
        
        break
        print()

1
1 황제 5 황혼 2 귀감 2 조율자 2 신비술사 2 명사수 1 추방자 1 닌자 
니달리 쓰레쉬 쉔 진 카시오페아 리븐 아지르 요네 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토', '여신의 눈물']
2
6 명사수 1 황제 2 황혼 2 신비술사 1 추방자 
니달리 베인 티모 징크스 진 카시오페아 아지르 요네 질리언 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '곡궁', '거인의 허리띠', 'B.F. 대검', '음전자 망토', '여신의 눈물']
3
1 우두머리 4 암살자 3 달빛 3 나무정령 2 싸움꾼 2 현혹술사 1 닌자 
마오카이 다이애나 리산드라 파이크 아칼리 카타리나 세트 이즈리얼 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토']
4
6 신비술사 1 황제 3 요술사 2 황혼 2 현혹술사 1 닌자 
잔나 럭스 유미 쉔 카시오페아 릴리아 아지르 이즈리얼 질리언 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토', '여신의 눈물']
5
1 우두머리 1 황제 6 대장군 3 귀감 2 암살자 2 싸움꾼 
니달리 바이 자르반 4세 파이크 카타리나 리븐 세트 아지르 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '거인의 허리띠', 'B.F. 대검', '음전자 망토', '여신의 눈물']
6
1 우두머리 3 사냥꾼 3 나무정령 3 요술사 2 조율자 2 싸움꾼 2 신비술사 1 추방자 1 닌자 
룰루 쉔 워윅 애쉬 릴리아 요네 질리언 세트 
['쓸데없이 큰 지팡이', '쇠사슬 조끼', '연습용 장갑', '곡궁', '거인의 허리띠', '뒤집개', 'B.F. 대검', '음전자 망토', '여신의 눈물']
7
8 결투가 2 추방자 1 황제 3 신성 
피오라 야스오 잭스 신 짜오 칼리스타 질리언 리 신 

In [13]:
a = [1,2,3]
lambda a : a+2

<function __main__.<lambda>(a)>

In [181]:
# driver = wd.Chrome(executable_path="/mnt/c/Users/shurona/Documents/chromedriver.exe")
# url = 'https://lolchess.gg/decks'
# driver.get(url)
# driver.find_element_by_css_selector('.decks-header__button').click()
# time.sleep(5)
# html = driver.page_source